In [4]:
import os

import pandas as pd
import torch
from torch import nn
import torchvision
from sysidentpy.metrics import mean_squared_error, root_relative_squared_error
from sysidentpy.neural_network import NARXNN
from sysidentpy.basis_function._basis_function import Polynomial, Fourier

In [5]:
import sklearn.model_selection as modsel    
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Get a data sample from the dataset in which pump 4 is running and resample 
# it to reduce the training time of the model
data = pd.read_parquet('Data/pump_station_data.parquet')
train_data = data['2023-02-14 10:00:00':'2023-02-14 11:20:00'].resample("10s").mean()


train_data = train_data.drop(labels=['height', 'pump1_power','pump4_power', 'pump1_rpm'], axis=1)
train_data = train_data.reset_index(drop=True)
scaled_data = scaler.fit_transform(train_data)


# Split the data into train and test data for the model training
train_data = pd.DataFrame(scaled_data,columns=train_data.columns, index=train_data.index)
data_train, data_test = modsel.train_test_split(train_data, test_size=0.2, shuffle=False)

# Format the data so that it can be used for training the model
x_train_4 = data_train['pump4_rpm'].values.reshape(-1,1)
y_train_4 = data_train['outflow'].values.reshape(-1,1)


# Format the data so that it can be used to fit the neural network
x_test_4 = data_test['pump4_rpm'].values.reshape(-1,1)
y_test_4 = data_test['outflow'].values.reshape(-1,1)

# Get test data for the model comparison after training 
# This data is taken just after the data used for training
test_data = data['2023-02-14 11:20:00':'2023-02-14 12:20:00'].resample("10s").mean()
test_data = test_data.drop(labels=['height', 'pump1_power','pump4_power', 'pump1_rpm'], axis=1)
test_data = test_data.reset_index(drop=True)
scaled_test_data = scaler.fit_transform(test_data)
test_data = pd.DataFrame(scaled_test_data,columns=test_data.columns, index=test_data.index)

#Format the data so that it can be used with the toolbox
x_test_compare_4 = test_data['pump4_rpm'].values.reshape(-1,1)
y_test_compare_4 = test_data['outflow'].values.reshape(-1,1)

# Get compare data from a day later to validate the model
compare_data = data['2023-02-15 11:00:00':'2023-02-15 13:00:00'].resample("10s").mean()

compare_data = compare_data.drop(labels=['height', 'pump1_power','pump4_power', 'pump1_rpm'], axis=1)
compare_data = compare_data.reset_index(drop=True)
scaled_compare_data = scaler.fit_transform(compare_data)
compare_data = pd.DataFrame(scaled_compare_data,columns=compare_data.columns, index=compare_data.index)
# X and Y values for the validation of the created model
x_compare_4 = compare_data['pump4_rpm'].values.reshape(-1,1)
y_compare_4 = compare_data['outflow'].values.reshape(-1,1)

In [12]:
basis_function = Polynomial(degree=1)
narx_net_1 = NARXNN(
                    ylag=3,
                    xlag= 11,
                    basis_function=basis_function,
                    model_type="NARMAX",
                    loss_func="mse_loss",
                    optimizer="Adam",
                    epochs=5000,
                    device='cuda',
                    learning_rate = 4e-3,
                    verbose=False,
                    optim_params={
                        "betas": (0.9, 0.999),
                        "eps": 1e-05,
                    }, 
                )
class NARX(nn.Module):
            def __init__(self):
                super().__init__()
                self.lin = nn.Linear((14), 30)
                self.lin2 = nn.Linear(30, 64)
                self.lin3 = nn.Linear(64, 64)
                self.lin4 = nn.Linear(64, 64)
                self.lin5 = nn.Linear(64, 30)               
                self.lin6 = nn.Linear(30, 1)
                self.tanh = eval("nn."+'RReLU'+"()")
                

            def forward(self, xb):
                z = self.lin(xb)
                z = self.tanh(z)
                z = self.lin2(z)
                z = self.tanh(z)
                z = self.lin3(z)
                z = self.tanh(z)
                z = self.lin4(z)
                z = self.tanh(z)
                z = self.lin5(z)
                z = self.tanh(z)
                z = self.lin6(z)
                z = self.tanh(z)
                
                return z
narx_net_1.net = NARX()
my_model = torch.load("best_model.pt")

narx_net_1.net.load_state_dict(my_model)

narx_net_1.net.eval()


predict = narx_net_1.predict(X=x_compare_4, y=y_compare_4, forecast_horizon=400)

TypeError: 'NoneType' object is not iterable